4.4 Serialization and Composition

1. How to seperate the prompt from the code


In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler

from langchain.prompts import load_prompt

#prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

prompt.format(country="Vietnam")

'what is the capital of Vietnam'

2. How to compose the prompts altogether

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts=[ # leftone should be same with the template name
    ("intro", intro),
    ("example", example),
    ("start", start)
]


full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)
# full_prompt.format(
#     character="Suspicious Mafia pretend to be a normal citizen",
#     example_question="What did you do last night",
#     example_answer = """
#     Oh, just a quiet evening at home. 
#     Just me, a glass of wine, and a little... paperwork to sort out. 
#     Nothing too exciting—just the usual stuff. 
#     How about you? Anything interesting?:)
#     """,
#     question="I think you are a bit suspicious. what is your secret?"
# )

chain = full_prompt | chat

chain.invoke({
    "character":"Suspicious Mafia pretend to be a normal citizen",
    "example_question":"What did you do last night",
    "example_answer" : """
    Oh, just a quiet evening at home. 
    Just me, a glass of wine, and a little... paperwork to sort out. 
    Nothing too exciting—just the usual stuff. 
    How about you? Anything interesting?:)
    """,
    "question":"I think you are a bit suspicious. what is your secret?"
})



Oh, me? Suspicious? No way, I'm just a regular citizen going about my business. 
I don't have any secrets, just living a simple life, you know? 
But hey, if you have any suspicions, feel free to share. I'm an open book. ;)

AIMessage(content="Oh, me? Suspicious? No way, I'm just a regular citizen going about my business. \nI don't have any secrets, just living a simple life, you know? \nBut hey, if you have any suspicions, feel free to share. I'm an open book. ;)", response_metadata={'finish_reason': 'stop'}, id='run-cf9031ca-c559-4a47-bb8d-f249d334fbe7-0')